<a href="https://colab.research.google.com/github/AlexeyTri/SemMed_fall25/blob/main/Seminar1/SemMed1_fall25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предобработка данных и функции потерь в линейной регрессии

## Данные
Для демонстраций загрузим набор данных https://archive.ics.uci.edu/dataset/10/automobile. В данных присутствуют категориальные, целочисленные и вещественнозначные признаки.

In [1]:
import pandas as pd

In [2]:
X_raw = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data", header=None, na_values=["?"])

## Предобработка данных

Предобработка данных важна при применении любых методов машинного обучения, а в особенности для линейных моделей. В sklearn предобработку удобно делать с помощью модуля preprocessing или методов библиотеки pandas.

In [9]:
from sklearn import impute

## Заполнение пропусков
В матрице объекты-признаки могут быть пропущенные значения, и это вызовет исключение при попытке передать такую матрицу в функцию обучения модели или даже предобработки. Если пропусков немного, можно удалить объекты с пропусками из обучающей выборки. Заполнить пропуски можно разными способами:

заполнить средними (mean, median);
предсказывать пропущенные значения по непропущенным.
Последний вариант сложный и применяется редко. Для заполнения константами можно использовать метод датафрейма fillna, для замены средними - класс preprocessing.Imputer.

## Преобразование нечисловых признаков

Практически все методы машинного обучения требуют, чтобы на вход функции обучения подавалась вещественная матрица. В процессе обучения используются свойства вещественных чисел, в частности, возможность сравнения и применения арифметических операций. Поэтому, даже если формально в матрице объекты-признаки записаны числовые значения, нужно всегда анализировать, можно ли относиться к ним как к числам.

Пример: некоторые признаки могут задаваться целочисленными хешами или id (например, id пользователя соц. сети), однако нельзя сложить двух пользователей и получить третьего, исходя из их id (как это может сделать линейная модель).

Это пример категориального признака, принимающего значения из неупорядоченного конечного множества
. К таким признакам обычно применяют one-hot encoding (вместо одного признака создают
 бинарных признаков - по одному на каждое возможное значение исходного признака). В sklearn это можно сделать с помощью классов LabelEncoder + OneHotEncoding, но проще использовать функцию pd.get_dummies.

Следует заметить, что в новой матрице будет очень много нулевых значений. Чтобы не хранить их в памяти, можно задать параметр OneHotEncoder(sparse = True) или .get_dummies(sparse=True), и метод вернет разреженную матрицу, в которой хранятся только ненулевые значения. Выполнение некоторых операций с такой матрицей может быть неэффективным, однако большинство методов sklearn умеют работать с разреженными матрицами.

Вопрос: какая проблема возникнет при применении такого способа кодирования для обучения линейной регрессии?

Необходимо удалить один из столбцов, созданных для каждого признака. Для этого в get_dummies надо поставить drop_first=True.

## Масштабирование признаков
При начале работы с данными всегда рекомендуется приводить все признаки к одному масштабу. Это важно по нескольким причинам:

ускорение обучения модели (пояснение на лекции);
улучшение численной устойчивости при работе с матрицей объекты-признаки (рядом с нулем чисел с плавающей точкой больше, чем с области больших чисел)
для линейных моделей: интерпретация весов при признаках как меры их значимости.
Первый популярный способ масштабирования - нормализация: вычитание среднего из каждого признака и деление на стандартное отклонение (StandardScaler в sklearn). Второй популярный способ: вычитание минимума из каждого признака, а затем деление на разницу максимального и минимального значения (MinMaxScaler в sklearn).

In [34]:
from sklearn import preprocessing


## Обучение модели и оценка качества

In [35]:
from sklearn.model_selection import train_test_split



In [36]:
from sklearn.linear_model import LinearRegression



In [37]:
from sklearn.metrics import r2_score


## Сильно переобучились.

Добавим регуляризацию:

L1-регуляризация: Lasso
L2-регуляризация: Ridge
L1+L2: ElasticNet

In [38]:
from sklearn.linear_model import Ridge


In [39]:
from sklearn.linear_model import SGDRegressor


In [40]:
from sklearn.model_selection import GridSearchCV
